### Creating a Reporting Dashboard using Dash
#### Arina Sitnikova

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from millify import prettify
from datetime import datetime as dt

Loading data and converting all column names to lowercase for convenience:

In [2]:
df = pd.read_csv('claims_test.csv')
df.columns = map(str.lower, df.columns)

There seem to be some NA values which we need to handle before creating a dashboard:

In [3]:
df.head()

,month,service_category,claim_specialty,payer,paid_amount
0,201801,AncillaryFFS,NaN,Payer F,128438
1,201801,AncillaryFFS,NaN,Payer H,66624
2,201801,AncillaryFFS,NaN,Payer O,118108
3,201801,AncillaryFFS,NaN,Payer W,1073
4,201801,AncillaryFFS,ACH,Payer W,604


In addition, some of the categorical variables can be transformed. That is, 'SpecialistsFFS' and 'SpecialistFFS' are highly likely to belong to the same category. I will address that further.

In [4]:
df['service_category'].value_counts()

SpecialistsFFS        12296
AncillaryFFS           9701
InpatientServices      9424
OutpatientServices     7582
ERServices             3696
ASCServices            2643
SNFServices            2501
SpecialistFFS          1684
PCPFFS                 1403
PCPEncounter           1222
Name: service_category, dtype: int64

To normalize text values, I followed a standard procedure: converting a string to lowercase, removing non-alphabetical characters, punctuation and extra white spaces.  

In [5]:
def normalize(text):
    text = text.str.lower()
    text = text.str.replace('\t', '')
    text = text.str.replace('\W', ' ')
    text = text.apply(lambda x: ' '.join(str(x).split()))
    return text

In addition, the preprocessing part included converting the 'month' variable into the datetime format, transforming several categorical variables (such as the one I mentioned above) and dropping some NA values:

In [6]:
def preprocessing(df):
    df['year'] = df['month'].astype(str).str[:4].astype(int)
    df['month_only'] = df['month'].astype(str).str[4:].astype(int)
    df = df[(df['month_only'] > 0) & (df['month_only'] < 13)] #there was a month with a value of 0
    df['date'] = df['month'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
    df['service_category'] = np.where(df['service_category'] == 'SpecialistFFS', 'SpecialistsFFS', df['service_category'])
    df = df.dropna(subset=['claim_specialty'])
    df['claim_specialty'] = normalize(df['claim_specialty'])
    df = df[df['paid_amount'] > 0]
    df = df.drop(['month', 'year', 'month_only'], axis = 1)
    return df

In [7]:
df = preprocessing(df)
df.head() #looks much better now!

,service_category,claim_specialty,payer,paid_amount,date
4,AncillaryFFS,ach,Payer W,604,2018-01-01
6,AncillaryFFS,advanced rn pract,Payer CA,1772,2018-01-01
7,AncillaryFFS,ambulance,Payer CA,446698,2018-01-01
8,AncillaryFFS,ambulance,Payer CO,51664,2018-01-01
9,AncillaryFFS,ambulance,Payer F,1083232,2018-01-01


Now I am ready to create a web dashboard. For that purpose, I decided to use Plotly/Dash.

In [9]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objs as go

My final dashboard contains three figures: two charts and one table. The top-left chart allows us to understand how much money each service category has brought in total since the beginning of 2018. For example, we can easily notice that FFS Specialists are by far the most high-paying compared to other services. It is also possible to filter data by categories so we won't lose any small numbers whatsoever.

The top-right table provides a better insight into the specialty claims. A dropdown gives us a possibility to explore a wide range of specialty claims. Each selection will help us find out which service categories are more in demand when it comes to some particular claim - based on the total number of claims as well as the total amount of money each service received.

Finally, the graph on the bottom shows how much different payers have contributed in the past 2.5 years. The graph has an option of filering data based on both date/period of time and payers.

In [10]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

def create_dict_list_of_service():
    dictlist = []
    unique_list = df['service_category'].unique()
    for service in unique_list:
        dictlist.append({'value': service, 'label': service})
    return dictlist

dict_services = create_dict_list_of_service()

def create_dict_list_of_payers():
    dictlist = []
    unique_list = df['payer'].unique()
    for payer in unique_list:
        dictlist.append({'value': payer, 'label': payer})
    return dictlist

dict_payers = create_dict_list_of_payers()

def create_dict_list_of_claims():
    dictlist = []
    unique_list = df['claim_specialty'].unique()
    for claim in unique_list:
        dictlist.append({'value': claim, 'label': claim})
    return dictlist

dict_claims = create_dict_list_of_claims()

app.layout = html.Div([
    #Layout and components of the bar chart
    html.Div([
        html.H4('Total amount paid, by service category'),
        dcc.Dropdown(
            id = 'service-dropdown',
            options = dict_services,
            multi = True,
            value = list(df.service_category.unique())
        ),
        dcc.Graph(
            id='service-bar'
        )
    ], style={'width': '48%', 'display': 'inline-block'}),
    
    ##Layout and components of the html table
    html.Div([
        html.H4('Focusing on the specialty claims'),
        html.P(''),
        dcc.Markdown('*Please select a claim below:*'),
        dcc.Dropdown(
            id = 'claims-dropdown',
            options = dict_claims,
            value = 'anesthesiology',
        ),        
        html.Table(id = 'claims-table'),
        html.P(''),
    ], style = {'width': '48%', 'float': 'right', 'display': 'inline-block'}),

    ##Layout and components of the line chart
    html.Div([
        html.H4('Total amount paid by selected payers, 2018-2020'),
        dcc.Dropdown(
            id = 'payer-dropdown',
            options = dict_payers,
            multi = True,
            value = list(df.payer.unique())
        ),
        #parameter to filter data by date range
        dcc.DatePickerRange(
            id = 'my-date-picker-range',
            min_date_allowed = dt(2018, 1, 1),
            max_date_allowed = dt(2050, 1, 1),
            initial_visible_month=dt(2018, 1, 1),
            #end_date=dt(2020, 7, 1).date(),
            start_date_placeholder_text = 'MM/DD/YYYY'
        ),
        dcc.Graph(
            id = 'timeseries', animate = True
        ), html.P(''),
    ], style={'width': '100%', 'display': 'inline-block'})
])

#Bar chart
@app.callback(Output('service-bar', 'figure'), [Input('service-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    df_filtered = df[(df['service_category'].isin(selected_dropdown_value))]
    
    group2 = df_filtered.groupby('service_category')['paid_amount'].sum().reset_index()
    group2.index = group2['service_category']
    
    figure = {
        'data': [go.Bar(
            x = sorted(group2.paid_amount),
            y = group2.index,
            orientation = 'h'
        )],
        'layout':go.Layout(
            yaxis = dict(
                autorange=True,
                automargin=True
            ),
            xaxis = dict(
                autorange=True,
                automargin = True,
                title = 'Total amount, $'
            )
        )
    }
    return figure    

#Line chart with DatePickerRange & slicers
@app.callback(Output('timeseries', 'figure'), [Input('payer-dropdown', 'value'),
                                               Input('my-date-picker-range', 'start_date'),
                                               Input('my-date-picker-range', 'end_date')])

def update_graph(selected_dropdown_value, start_date, end_date):
    group1 = df.groupby(['payer', 'date'])['paid_amount'].sum().reset_index()
    group1.index = pd.to_datetime(group1['date'])
    
    trace = []
    df_sub = group1
    
    if start_date is not None:
        df_sub = df_sub.loc[start_date:]
    
    if end_date is not None:
        df_sub = df_sub.loc[:end_date]
    
    for payer in selected_dropdown_value:
        trace.append(go.Scatter(x=df_sub[df_sub['payer'] == payer]['date'],
                                 y=df_sub[df_sub['payer'] == payer]['paid_amount'],
                                 mode='lines',
                                 opacity=0.7,
                                 name=payer,
                                 textposition='bottom center'))
    traces = [trace]
    data = [val for sublist in traces for val in sublist]


    figure = {'data': data,
              'layout': go.Layout(
                  colorway=["#5E0DAC", '#FF4F00', '#375CB1', '#FF7400', '#FFF400', '#FF0056'],
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  xaxis={'range': [df_sub.index.min(), df_sub.index.max()]}
              )}
    
    return figure

#Html table
@app.callback(Output('claims-table', 'children'), [Input('claims-dropdown', 'value')])
def generate_table(selected_dropdown_value, max_rows = 10):
    claims_filter = df[(df['claim_specialty'] == selected_dropdown_value)]
    
    service_count = pd.DataFrame(claims_filter['service_category'].value_counts()).reset_index()
    service_count.columns = ['service_category', 'claims_number']
    
    frequent_payer = pd.DataFrame(claims_filter.groupby('service_category')['payer'].apply(lambda x: x.value_counts().head(1))).reset_index()
    frequent_payer.columns = ['service_category', 'most_frequent_payer', 'total_no_payer']
    frequent_payer = frequent_payer.drop('total_no_payer', axis = 1)
    
    service_pay = claims_filter.groupby('service_category')['paid_amount'].sum().reset_index()
    
    some_df = service_pay.merge(service_count.merge(frequent_payer)).sort_values(['paid_amount', 'claims_number'], ascending = False)
    some_df.columns = ['Service category', 'Total amount paid', 'Total number of claims', 'Most frequent payer']
    some_df['Total amount paid'] = some_df['Total amount paid'].apply(lambda x: '$' + str(prettify(x)))
    
    return html.Table([html.Tr([html.Th(col) for col in some_df.columns])] + [html.Tr([
        html.Td(some_df.iloc[i][col]) for col in some_df.columns
    ]) for i in range(min(len(some_df), max_rows))])


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Sep/2020 16:04:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16:04:12] "GET /_dash-component-suites/dash_renderer/react@16.v1_7_0m1598690383.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16:04:12] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_7_0m1598690383.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16:04:12] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_7_0m1598690383.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16:04:12] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_7_0m1598690383.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16:04:12] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_11_0m1598690383.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16:04:12] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_0m1598690383.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2020 16